In [1]:
#1 importing libraries and methodes
from bs4 import BeautifulSoup
import requests,re
from bs4 import BeautifulSoup as bs
import json

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

In [3]:
#2 inputting the url
url='https://www.immoweb.be/en/classified/apartment/for-sale/gavere/9890/9937664?searchId=62971b1f86bff'
# url1='https://www.immoweb.be/en/search/apartment/for-sale?countries=BE'

In [4]:
#Creat a request to get url
#parse HTML with beautifulSoup
r = requests.get(url)
# print(url, r.status_code)
soup = BeautifulSoup(r.content, "lxml")


In [5]:
#get tables from webpage
tables = []
tables = soup.find_all('tbody',class_='classified-table__body')
general_table = tables[0].find_all('tr')
interior_table = tables[1].find_all('tr')
exterior_table = tables[2].find_all('tr')
facilities_table = tables[3].find_all('tr')
financial_table=tables[6].find_all('tr')


In [6]:
#get the info from general table
dict_general={}
list_general_key=[]
list_general_value=[]

for row in general_table:
    for th in row.find_all('th'):
        print('key:',f'{th.contents[0].strip()}') 
        list_general_key.append(th.contents[0].strip())
        
    for td in row.find_all('td'):
        print('value:',f'{td.contents[0].strip()}')  
        list_general_value.append(td.contents[0].strip())
        

key: Available as of
value: At delivery
key: Available date
value: November 1 2023 - 12:00 AM
key: Neighbourhood or locality
value: Gent en omstreken
key: Construction year
value: 2021
key: Floor
value: 2
key: Number of floors
value: 2
key: Building condition
value: As new
key: Number of frontages
value: 4
key: Surroundings type
value: Urban


In [7]:
dict_general=dict(zip(list_general_key,list_general_value))


In [8]:
#get the info from interior table
dict_interior={}
list_interior_key=[]
list_interior_value=[]

for row in interior_table:
    for th in row.find_all('th'):
        print('key:',f'{th.contents[0].strip()}')
        list_interior_key.append(th.contents[0].strip())
        
    for td in row.find_all('td'):
        print('value:',f'{td.contents[0].strip()}') 
        list_interior_value.append(td.contents[0].strip())

key: Living area
value: 78
key: Living room surface
value: 15
key: Kitchen type
value: Hyper equipped
key: Kitchen surface
value: 7
key: Bedrooms
value: 1
key: Bedroom 1 surface
value: 18
key: Bathrooms
value: 1
key: Toilets
value: 1
key: Furnished
value: No


In [9]:
dict_interior=dict(zip(list_interior_key,list_interior_value))



In [10]:
#get the info from exterior table

dict_exterior={}
list_exterior_key=[]
list_exterior_value=[]

for row in exterior_table:
    for th in row.find_all('th'):
        print('key:',f'{th.contents[0].strip()}')
        list_exterior_key.append(th.contents[0].strip())
        
    for td in row.find_all('td'):
        print('value:',f'{td.contents[0].strip()}')
        list_exterior_value.append(td.contents[0].strip())
        

key: Terrace surface
value: 10


In [11]:
dict_exterior=dict(zip(list_exterior_key,list_exterior_value))


In [12]:
#get the info from facilities table

dict_facilities={}
list_facilities_key=[]
list_facilities_value=[]

for row in facilities_table:
    for th in row.find_all('th'):
        print('key:',f'{th.contents[0].strip()}')
        list_facilities_key.append(th.contents[0].strip())
        
    for td in row.find_all('td'):
        print('value:',f'{td.contents[0].strip()}')
        list_facilities_value.append(td.contents[0].strip())

key: Elevator
value: Yes
key: Accessible for disabled people
value: Yes
key: Visio phone
value: Yes
key: Swimming pool
value: No


In [13]:
dict_facilities=dict(zip(list_facilities_key,list_facilities_value))



In [14]:
#get the info from financial table

dict_financial={}
list_financial_key=[]
list_financial_value=[]

for row in financial_table:
    for th in row.find_all('th'):
        print('key:',f'{th.contents[0].strip()}')
        list_financial_key.append(th.contents[0].strip())
        
    for td in row.find_all('td'):
        if len(td.find_all('span'))>0:
            print(td.find_all('span')[0].contents[0].strip())
            list_financial_value.append(td.find_all('span')[0].contents[0].strip())
        else:
            print('value:',f'{td.contents[0].strip()}')
        # print('value:',f'{td.contents[0].strip()}')
        #list_financial_value.append(td.contents[0].strip())

key: Price
€ 220,000
key: Cadastral income
€ 1
key: Tenement building
value: No
value: 


In [15]:
dict_financial=dict(zip(list_financial_key,list_financial_value))


In [16]:
#put all the dictionaries together
dict_all = {**dict_general,**dict_interior, **dict_exterior, **dict_facilities, **dict_financial}


In [17]:
#get the zip from the url with regex 
zipcode = re.findall("([/]+[\d{4}]+[/])",str(url)) 
zipcode = zipcode[0].replace("/","")
    


In [18]:
#appen the zipcode to the dictionary
dict_all.update( [('zipcode', zipcode)] )
#get all the data in dictionary

In [19]:
#get in each page all the links of the houses 
#put the links in a list
url='https://www.immoweb.be/en/search/apartment/for-sale?countries=BE'
driver = webdriver.Firefox()
# driver.implicitly_wait(30)
driver.get(url)

product_urls = []

for page in range(1,4):
    url2=(f'https://www.immoweb.be/en/search/apartment/for-sale?countries=BE&page={page}&orderBy=relevance')
    product_info = driver.find_elements(by=By.XPATH,value="//a[@class='card__title-link']")
    for info in product_info:
        product_urls.append(info.get_attribute("href"))


In [20]:
#put the list of links in csv file
with open('houses.txt', 'w') as f:
    f.write(json.dumps(product_urls))

In [21]:
#create a dataframe from the dictionary
import pandas as pd
df = pd.json_normalize(dict_all)
df

,Available as of,Available date,Neighbourhood or locality,Construction year,Floor,Number of floors,Building condition,Number of frontages,Surroundings type,Living area,...,Toilets,Furnished,Terrace surface,Elevator,Accessible for disabled people,Visio phone,Swimming pool,Price,Cadastral income,zipcode
0,At delivery,November 1 2023 - 12:00 AM,Gent en omstreken,2021,2,2,As new,4,Urban,78,...,1,No,10,Yes,Yes,Yes,No,"€ 220,000",€ 1,9890


In [22]:
#put the dataframe in csv file
df.to_csv('data1.csv',index=False, header=True)
